## Citibike Analysis using SQL and Carto Plotting 

In [1]:
# Set Up API and Libraries

SQL_SOURCE = 'https://ixx200.carto.com/api/v2/sql?q='

# Python 2 AND 3 Compatibility Importing
try:
    from urllib.parse import urlparse, urlencode
    from urllib.request import urlopen, Request
    from urllib.error import HTTPError
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode
    from urllib2 import urlopen, Request, HTTPError
    
from io import StringIO
import ast
import pandas as pd


# Carto SQL API Set Up
def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    data = urlencode({'format': format, 'q': query}).encode('utf-8')
    try:
        response = urlopen(source, data)
    except HTTPError as e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return pd.read_csv(response)

---

### Task 1: Familiarize with SQL

Basic
- Sort data by start_station_id, tripduration
- Only checking trips with duration <= 3 hours
- Only show the top/last 10 records (aka head and tail in SQL)
- List all unique start_station_id values

Use Aggregation functions:
- Count the number of trips (aka wc -l in SQL)
- Find the average/min/max trip duration

In [2]:
# quick scan

t0 = '''
SELECT *
from citibike
limit 10
'''

queryCartoDB(t0)

,cartodb_id,the_geom,the_geom_webmercator,gender,birth_year,bikeid,end_station_id,start_station_id,tripduration,usertype,end_station_name,start_station_name,end_station_longitude,end_station_latitude,start_station_longitude,start_station_latitude,stoptime,starttime
0,5960,NaN,NaN,1,1965,19217,462,435,351,Subscriber,W 22 St & 10 Ave,W 21 St & 6 Ave,-74.004519,40.746920,-73.994156,40.741740,2015-02-01 20:42:00+00,2015-02-01 20:36:00+00
1,9806,NaN,NaN,1,1971,17575,488,167,643,Subscriber,W 39 St & 9 Ave,E 39 St & 3 Ave,-73.993722,40.756458,-73.976049,40.748901,2015-02-03 14:50:00+00,2015-02-03 14:39:00+00
2,13113,NaN,NaN,1,1973,20313,440,332,1379,Subscriber,E 45 St & 3 Ave,Cherry St,-73.972826,40.752554,-73.979481,40.712199,2015-02-04 07:30:00+00,2015-02-04 07:07:00+00
3,1,NaN,NaN,2,1978,17131,423,521,801,Subscriber,W 54 St & 9 Ave,8 Ave & W 31 St,-73.986905,40.765849,-73.994811,40.750450,2015-02-01 00:14:00+00,2015-02-01 00:00:00+00
4,2,NaN,NaN,1,1993,21289,504,497,379,Subscriber,1 Ave & E 15 St,E 17 St & Broadway,-73.981656,40.732219,-73.990093,40.737050,2015-02-01 00:07:00+00,2015-02-01 00:00:00+00
5,3,NaN,NaN,2,1969,18903,127,281,2474,Subscriber,Barrow St & Hudson St,Grand Army Plaza & Central Park S,-74.006744,40.731724,-73.973715,40.764397,2015-02-01 00:42:00+00,2015-02-01 00:01:00+00
6,4,NaN,NaN,2,1985,21044,505,2004,818,Subscriber,6 Ave & W 33 St,6 Ave & Broome St,-73.988484,40.749013,-74.004704,40.724399,2015-02-01 00:15:00+00,2015-02-01 00:01:00+00
7,5,NaN,NaN,1,1957,19868,83,323,544,Subscriber,Atlantic Ave & Fort Greene Pl,Lawrence St & Willoughby St,-73.976323,40.683826,-73.986317,40.692362,2015-02-01 00:10:00+00,2015-02-01 00:01:00+00
8,6,NaN,NaN,1,1979,15854,2002,373,717,Subscriber,Wythe Ave & Metropolitan Ave,Willoughby Ave & Walworth St,-73.963198,40.716887,-73.953820,40.693317,2015-02-01 00:14:00+00,2015-02-01 00:02:00+00
9,7,NaN,NaN,1,1983,15173,504,352,1306,Subscriber,1 Ave & E 15 St,W 56 St & 6 Ave,-73.981656,40.732219,-73.977225,40.763406,2015-02-01 00:26:00+00,2015-02-01 00:04:00+00


In [3]:
# return top 10 based on Station Id and Average Duration

t1 = '''
SELECT 
start_station_id, 
count(distinct start_station_id) as duplicate_cnt, 
avg(tripduration) as avg_duration, 
min(tripduration) as min_duration, 
max(tripduration) As max_duration
from citibike
where tripduration < 10800
group by start_station_id
order by start_station_id DESC, avg_duration DESC
limit 10
'''

queryCartoDB(t1)

,start_station_id,duplicate_cnt,avg_duration,min_duration,max_duration
0,3002,1,714.646739,79,7225
1,2023,1,771.978022,108,2454
2,2022,1,979.062500,107,2495
3,2021,1,738.815603,90,6592
4,2017,1,763.383721,109,2476
5,2012,1,714.519531,79,2026
6,2010,1,763.184211,167,2610
7,2009,1,777.794393,60,2765
8,2008,1,832.220930,94,3476
9,2006,1,1130.531646,90,3459


In [4]:
# return bottom 10 based on Station Id and Average Duration

t1 = '''
SELECT 
start_station_id, 
count(distinct start_station_id) as duplicate_cnt, 
avg(tripduration) as avg_duration, 
min(tripduration) as min_duration, 
max(tripduration) As max_duration
from citibike
where tripduration < 10800
group by start_station_id
order by start_station_id ASC, avg_duration ASC
limit 10
'''

queryCartoDB(t1)

,start_station_id,duplicate_cnt,avg_duration,min_duration,max_duration
0,72,1,742.701754,107,2099
1,79,1,659.140845,149,1916
2,82,1,445.346939,138,1581
3,83,1,783.685714,73,2647
4,116,1,543.965278,66,2081
5,119,1,1044.789474,278,2167
6,120,1,979.882353,270,2131
7,127,1,710.846890,83,5776
8,128,1,664.206478,62,2875
9,137,1,697.364583,85,2316


---

### Task 2

- Selecting trips started on Feb-02-2015 only
- Selecting trips started on the weekends
- What are average trip duration during weekends?
- Can we do the same for weekday?

In [ ]:
test = '''

SELECT max(starttime) as max, 
min(starttime) as min
from citibike

'''

queryCartoDB(test)

,max,min
0,2015-02-07 23:59:00+00,2015-02-01 00:00:00+00


In [ ]:
# average trip duration for trips on Feb 02, 2015

t2 = '''
SELECT 
'2016-02-02' as day,
avg(tripduration) as avg_tripduration
FROM citibike
WHERE starttime >= '2015-02-02 00:00:00' AND starttime <= '2015-02-02 23:59:59'
'''

queryCartoDB(t2)

In [ ]:
# Average Trip Duration during Weekends

t2 = '''
SELECT 
date_part('dow', starttime) as day_of_week, 
CASE
	WHEN date_part('dow', starttime) = 0 THEN 'Sunday'
    WHEN date_part('dow', starttime) = 1 THEN 'Monday'
    WHEN date_part('dow', starttime) = 2 THEN 'Tuesday'
    WHEN date_part('dow', starttime) = 3 THEN 'Wednesday'
    WHEN date_part('dow', starttime) = 4 THEN 'Thursday'
    WHEN date_part('dow', starttime) = 5 THEN 'Friday'
    WHEN date_part('dow', starttime) = 6 THEN 'Saturday'
END as Weekday,
avg(tripduration) as avg_tripDuration
FROM citibike
WHERE (date_part('dow', starttime) = 0 OR date_part('dow', starttime) = 6)
GROUP BY date_part('dow', starttime)
'''

queryCartoDB(t2)

In [ ]:
# Average Trip Duration during Week Days

t2 = '''
SELECT 
date_part('dow', starttime) as day_of_week, 
CASE
	WHEN date_part('dow', starttime) = 0 THEN 'Sunday'
    WHEN date_part('dow', starttime) = 1 THEN 'Monday'
    WHEN date_part('dow', starttime) = 2 THEN 'Tuesday'
    WHEN date_part('dow', starttime) = 3 THEN 'Wednesday'
    WHEN date_part('dow', starttime) = 4 THEN 'Thursday'
    WHEN date_part('dow', starttime) = 5 THEN 'Friday'
    WHEN date_part('dow', starttime) = 6 THEN 'Saturday'
END as Weekday,
avg(tripduration) as avg_tripDuration
FROM citibike
WHERE date_part('dow', starttime) NOT IN (0,6)
GROUP BY date_part('dow', starttime)
ORDER BY day_of_week ASC
'''

queryCartoDB(t2)

---

### Task 3

- Showing the list of start station locations
- Using GROUP BY
- Showing the number of trips started per station
- … but only for stations within 500m of Time Square!
- The coordinates of Time Square is (40.7577,-73.9857)

In [ ]:
t3 = '''
SELECT 
CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator, 
start_station_latitude, 
start_station_longitude,
MIN(cartodb_id) as cartodb_id, 
COUNT(tripduration) as numtrips
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                CDB_LatLng(40.7577, -73.9857)::geography,
                500) 
GROUP BY start_station_latitude, start_station_longitude
ORDER BY COUNT(tripduration) DESC

'''

queryCartoDB(t3)

<img style="float: left;", src="./screenshots/A2_1.png", width=600, height=600>

### Figure 1

This is the map output from the SQL above. It shows all stations within 500m near Time Square. The number label is the number of trips started from that station.

---

### Task 4

- Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!
- Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 

In [ ]:
t4 = '''

SELECT 
CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator, 
start_station_latitude, 
start_station_longitude,
MIN(cartodb_id) as cartodb_id, 
AVG(tripduration) as avg_tripduration

FROM citibike

WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
                CDB_LatLng(40.7577, -73.9857)::geography,
                500) AND
      (date_part('dow', starttime) = 0 OR date_part('dow', starttime) = 6)
      
GROUP BY start_station_latitude, start_station_longitude
ORDER BY AVG(tripduration) DESC
LIMIT 1
'''

queryCartoDB(t4)

In [ ]:
# Extra Credit

t4 = '''
SELECT ST_Makeline(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)),
                   CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude, end_station_longitude))) AS the_geom_webmercator,
	MIN(cartodb_id) AS cartodb_id,
    COUNT(tripduration) AS numTrips
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography,
                500)
	AND tripduration < 7200
GROUP BY start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude

'''

queryCartoDB(t4)

<img style="float: left;", src="./screenshots/A2_2.png", width=600, height=600>

### Figure 2

This is a map of all trips started from stations within 500m of Times Squares and lasted less than 2 hours. It is interesting to see where people go from Time Square and can travel within 2 hours (some get to downtown Brooklyn).